In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import gradio as gr

In [6]:
CATEGORY_RECOMMENDATIONS = {
    "Travel": "Consider reducing travel expenses by using more public transportation.",
    "Shopping": "Monitor your shopping habits to identify non-essential purchases.",
    "Food": "Try home-cooked meals to cut down on food expenses.",
    "Entertainment": "Limit entertainment subscriptions or look for free alternatives.",
    "Rent": "Evaluate if your rent fits your income; consider moving to a more affordable place.",
    "Health": "Allocate funds to health insurance or preventive care.",
    "Bills": "Optimize utility usage to reduce bills.",
    "Education": "Look for scholarships or financial aids to cut educational expenses."
}

In [12]:
def train_model(data):
    encoder = LabelEncoder()
    data['Category'] = encoder.fit_transform(data['Category'])

    # Preprocess text using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    X_text = vectorizer.fit_transform(data['Description'])

    # Combine TF-IDF with Amount
    X_amount = data['Amount'].values.reshape(-1, 1)
    X = np.hstack([X_text.toarray(), X_amount])

    # Target variable
    y = data['Category']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train SVM model
    model = SVC(kernel='linear', random_state=42)
    model.fit(X_train, y_train)

    return model, vectorizer, encoder

In [8]:
def generate_suggestions(category_summary, total_spent):
    suggestions = []

    if category_summary.get('Food', 0) > total_spent * 0.3:
        suggestions.append("You might be spending too much on food. Try cooking at home to save money.")
    if category_summary.get('Travel', 0) > total_spent * 0.3:
        suggestions.append("Consider reducing travel expenses by using more public transportation.")
    if category_summary.get('Entertainment', 0) > total_spent * 0.2:
        suggestions.append("Cut back on entertainment costs by seeking free events or discounts.")
    if category_summary.get('Shopping', 0) > total_spent * 0.4:
        suggestions.append("You could reduce shopping expenses by setting a monthly budget for clothes and gadgets.")
    if category_summary.get('Bills', 0) > total_spent * 0.3:
        suggestions.append("Try lowering your bills by comparing service providers for better rates.")
    if category_summary.get('Rent', 0) > total_spent * 0.4:
        suggestions.append("Evaluate if your rent fits your income. Consider more affordable housing options.")
    if category_summary.get('Health', 0) > total_spent * 0.1:
        suggestions.append("Monitor health-related expenses. Ensure they are necessary and explore preventive care options.")
    if category_summary.get('Education', 0) > total_spent * 0.2:
        suggestions.append("Look for scholarships, financial aid, or affordable online courses to reduce education expenses.")

    return suggestions


In [36]:
def process_expenses(file):
    try:
        # Read the uploaded CSV file
        data = pd.read_csv(file)

        # Check required columns
        if not all(col in data.columns for col in ['Description', 'Amount', 'Category']):
            return "Error: CSV must contain 'Description', 'Amount', and 'Category' columns."

        # Train the model
        model, vectorizer, encoder = train_model(data)

        data['Decoded_Category'] = encoder.inverse_transform(data['Category'])

        # Summarize monthly expenses
        summary = data.groupby('Decoded_Category')['Amount'].sum()
        total_expenses = summary.sum()

        category_summary = summary.to_dict()
        suggestions = generate_suggestions(summary, total_expenses)

        category_counts = data['Decoded_Category'].value_counts()
        summary_with_counts =summary.index.to_series().apply(
            lambda x: f"{x:<20} - {category_counts[x]} entries")

        category_details = "\n".join([f"{cat}: {amount}$" for cat, amount in summary.items()])

        suggestion_text = "Suggestions:\n" + "\n".join(suggestions)

        return f"{summary_with_counts.to_string(index=False, header=False).replace('  ','')}\n\n{category_details}\n\n{suggestion_text}"

    except Exception as e:
        return f"Error processing file: {e}"


In [57]:
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
from PIL import Image

# Function to process expenses and generate visualizations
def process_expenses2(file):
    try:
        # Read the uploaded CSV file
        data = pd.read_csv(file)

        # Check required columns
        if not all(col in data.columns for col in ['Description', 'Amount', 'Category']):
            return "Error: CSV must contain 'Description', 'Amount', and 'Category' columns.", None, None

        # Generate category counts (how many entries for each category)
        category_counts = data['Category'].value_counts()

        # Summarize monthly expenses
        summary = data.groupby('Category')['Amount'].sum()
        total_expenses = summary.sum()

        # Create summary with category count and amount
        summary_with_counts = summary.index.to_series().apply(
            lambda x: f"{x:<20} - {category_counts[x]} entries"
        )

        suggestions = generate_suggestions(summary, total_expenses)

        # Format category details (only category and amount)
        category_details = "\n".join([f"{cat}: {amount}$" for cat, amount in summary.items()])

        # Format suggestions
        suggestion_text = "Suggestions:\n" + "\n".join(suggestions)

        # Create Bar Graph for Expense Distribution
        plt.figure(figsize=(10, 6))
        sns.barplot(x=summary.index, y=summary.values)
        plt.title('Monthly Expense Distribution')
        plt.xlabel('Category')
        plt.ylabel('Amount ($)')

        # Save to a BytesIO object to pass in Gradio
        bar_graph_io = io.BytesIO()
        plt.savefig(bar_graph_io, format='png')
        bar_graph_io.seek(0)
        bar_graph_image = Image.open(bar_graph_io)
        plt.close()

        # Create Pie Chart for Expense Distribution
        plt.figure(figsize=(8, 8))
        plt.pie(summary.values, labels=summary.index, autopct='%1.1f%%', startangle=90)
        plt.title('Monthly Expense Breakdown')

        # Save to a BytesIO object to pass in Gradio
        pie_chart_io = io.BytesIO()
        plt.savefig(pie_chart_io, format='png')
        pie_chart_io.seek(0)
        pie_chart_image = Image.open(pie_chart_io)
        plt.close()

        # Combine all output into a final string for display
        output = f"{summary_with_counts.to_string(index=False, header=False).replace('  ', '')}\n\n{category_details}\n\n{suggestion_text}"

        # Return the visualizations and text output
        return output, bar_graph_image, pie_chart_image

    except Exception as e:
        return f"Error processing file: {e}", None, None

In [58]:
# Gradio interface
interface = gr.Interface(
    fn=process_expenses2,
    inputs=gr.File(label="Upload CSV File"),
    outputs=[ gr.Textbox(label="Expense Summary and Suggestions"),
        gr.Image(label="Expense Distribution Bar Chart"),
        gr.Image(label="Expense Breakdown Pie Chart")],  # Text output + 2 images (pie chart & bar chart)
    title="AI-Powered Expense Tracker",
    description="Upload a CSV file with 'Description', 'Amount', and 'Category' columns to analyze your monthly expenses and get suggestions."
)

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b55774757360185328.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
